In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class mLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, output_size):
        super(mLSTM, self).__init__()

        self.hidden_size = hidden_size
        # input embedding
        self.encoder = nn.Embedding(input_size, embed_size)
        # lstm weights
        self.weight_fm = nn.Linear(hidden_size, hidden_size)
        self.weight_im = nn.Linear(hidden_size, hidden_size)
        self.weight_cm = nn.Linear(hidden_size, hidden_size)
        self.weight_om = nn.Linear(hidden_size, hidden_size)
        self.weight_fx = nn.Linear(embed_size, hidden_size)
        self.weight_ix = nn.Linear(embed_size, hidden_size)
        self.weight_cx = nn.Linear(embed_size, hidden_size)
        self.weight_ox = nn.Linear(embed_size, hidden_size)
        # multiplicative weights
        self.weight_mh = nn.Linear(hidden_size, hidden_size)
        self.weight_mx = nn.Linear(embed_size, hidden_size)
        # decoder
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inp, h_0, c_0):
        # encode the input characters
        inp = self.encoder(inp)
        # calculate the multiplicative matrix
        m_t = self.weight_mx(inp) * self.weight_mh(h_0)
        # forget, input and output gates
        f_g = torch.sigmoid(self.weight_fx(inp) + self.weight_fm(m_t))
        i_g = torch.sigmoid(self.weight_ix(inp) + self.weight_im(m_t))
        o_g = torch.sigmoid(self.weight_ox(inp) + self.weight_om(m_t))
        # intermediate cell state
        c_tilda = torch.tanh(self.weight_cx(inp) + self.weight_cm(m_t))
        # current cell state
        cx = f_g * c_0 + i_g * c_tilda
        # hidden state
        hx = o_g * torch.tanh(cx)

        out = self.decoder(hx.view(1,-1))

        return out, hx, cx

    def init_hidden(self):
        h_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        c_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        return h_0, c_0



In [2]:
# Converte string para uma lista de inteiros
from unidecode import unidecode
import random

def generate_hidden(text, rnn):
    hidden, cell = rnn.init_hidden()
    text_tensor = char_tensor(text).cuda()
    for p in range(len(text)):
        output, hidden, cell = rnn(text_tensor[p], hidden, cell)
    return hidden, cell

def char_tensor(string):
    string = unidecode(string)
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = ord(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('The omega (Ω) symbol\n'))  

tensor([ 84, 104, 101,  32, 111, 109, 101, 103,  97,  32,  40,  79,  41,  32,
        115, 121, 109,  98, 111, 108,  10])


In [3]:
embed_size = 128 # ascii representation
hidden_size = 2048

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()
        self.rnn.load_state_dict(torch.load("lstm_11.pth"))
        self.fc1 = nn.Linear(2*hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, int(hidden_size/4))
        self.fc3 = nn.Linear(int(hidden_size/4), int(hidden_size/2))
        self.fc4 = nn.Linear(int(hidden_size/2), 5)
        self.dropout = nn.Dropout(0.5) 
        
    def forward(self, text_tensor):
        with torch.no_grad():
            hidden, cell = self.rnn.init_hidden()
            for p in range(len(text_tensor)):
                _, hidden, cell = self.rnn(text_tensor[p], hidden, cell)
        
        
        x = torch.cat((hidden, cell), 0).view(1, -1)
        x = F.relu(self.dropout(self.fc1(x)))
        x = F.relu(self.dropout(self.fc2(x)))
        x = F.relu(self.dropout(self.fc3(x)))
        x = F.relu(self.fc4(x))
        return x

In [4]:
import pandas as pd
df_test = pd.read_csv("../data/interim/test.csv")

In [5]:
test_data = dict()
for score, message in zip(df_test.score.values, df_test.message.values):
    try:
        test_data[score].append([score, message])
    except:
        test_data[score] = [[score, message]]

In [6]:
net = Net()
net.load_state_dict(torch.load("balanced_net_9.pth"))
net = net.cuda()
net.eval()

Net(
  (rnn): mLSTM(
    (encoder): Embedding(128, 128)
    (weight_fm): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_im): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_cm): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_om): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_fx): Linear(in_features=128, out_features=2048, bias=True)
    (weight_ix): Linear(in_features=128, out_features=2048, bias=True)
    (weight_cx): Linear(in_features=128, out_features=2048, bias=True)
    (weight_ox): Linear(in_features=128, out_features=2048, bias=True)
    (weight_mh): Linear(in_features=2048, out_features=2048, bias=True)
    (weight_mx): Linear(in_features=128, out_features=2048, bias=True)
    (decoder): Linear(in_features=2048, out_features=128, bias=True)
  )
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
  (fc3): Linear(in_features=5

In [7]:
from tqdm import tqdm
pred, y, messages = list(), list(), list()
for score in test_data:
    for _, message in tqdm(test_data[score], total=len(test_data[score])):
        y.append(score)
        messages.append(message)
        pred.append(net(char_tensor(message).cuda()))

100%|██████████| 698/698 [01:01<00:00,  9.92it/s]


In [8]:
def get_score(prediction):
    hiscore, higrade = 1, 0
    for score, e in enumerate(list(prediction.detach()[0])):
        if e.item() > higrade:
            hiscore = score+1
            higrade = e.item()
    return hiscore#, higrade

In [9]:
pred = list(map(get_score, pred))

In [10]:
correct = incorrect = 0
for p, y_value, message in zip(pred, y, messages):
    if p == y_value:
        correct += 1
    else:
        incorrect += 1
        print(p, y_value, message)

1 5 Ótimo produto e chegou bem rápido.
1 5 Otima loja,, recomendo,...
1 5 Gostei muito pelo comprimento do tempo de entrega,bem rapido 
Estão de parabens
1 5 Recebi o produto antes do previsto, td em ordem Recomendo. 
1 5 Recebi antes do prazo, muito rápio.
1 5 Excelente! 
Veio certinho, veio bem antes do dia. 
Super recomendo... 
Obg 
1 5 Ótimo preço eproduto
1 5 Recebi o produto antes do prazo e corretamente.
1 5 Muito bom e rapido
1 5 Adorei, compro novamente!
1 5 Produto de boa qualidade, recebi dentro do prazo - recomendo.
1 5 Preciso devolver o produto pois essa compra havia sido cancelada. Acredito que tenha sido problema do sistema. Estou aguardando resposta para devolver e pegar o dinheiro de volta
1 5 Lindo, grande, brilhante de excelente qualidade e como sempre chegou bem antes do prazo...Super recomendo ? 
1 5 Recomendo, entrega super rápida 
1 5 Gostei da experiência o produto conforme o anunciado, bonito as medições corretas.
Muito satisfeita. 
1 5 Produto atendeu as expe

1 5 Prazo de entrega para 9 dias, chegou no 2o.!!!
1 5 Otima e chegou em bom estado .. Não substituí a prancha mas ajuda muito no dia a dia .. Como não gosto de pranchar pq acho chato ela me ajudou mt 
1 5 TUDO CERTO ENTREGUE NO PRAZO
1 5 recebemos o produto.
1 5 Otimo preço e uma facilidade em poder localizar no site. 
1 5 Preço e produto muito bons. Atendimento rapido e entrega antes do przo
1 5 Ótimo produto... ótimo vendedor!!
1 5 Produto dentro do prazo de entrega e também conforme feito pedido.
1 5 Produto certinho, entrega antes do tempo. 
1 5 Tudo perfeito com a transação.
1 5 Entregaram antes do prazo 😀
1 5 Novo, lacrado, otimo atendimento, recomendo
1 5 A descrição do produto está muito confuso....SEGUE DETALHAMENTO......
aíses Europeus Controle De Ipod Traseiro, Pode Carregar O Ipod/Iphone Rca Câmera De Visão Traseira De Entrada Relógio Em Tempo Re
1 5 Chegou antes do prazo informado.Isso demonstra confiança no vendedor
1 5 Produto como o da foto e chegou super rápido!
1 5 A

1 5 Perfeito... entrega antes do previsão 
Produto otimoo
1 5 Entrega muita rápida, e o patins é bom e muito bonito. Único problema foi os parafusos muito apertados.
1 5 Sim, já comprei e sempre compro, estou satisfeito com os produtos e seus prazos de entrega. 
1 5 Ótima loja, entregou o produto corretamente antes do término do prazo.
1 5 Excelente qualidade 
1 5 É muito lindo! chegou bem embalado e rapidinho. Adorei! 
1 5 Produto excelente, TOP - Vale muitoooo a pena
1 5 O produto foi entregue super rápido, bem antes da data prevista.
1 5 Tudo ótimo 
1 5 EXCELENTE PRODUTO, ESPAÇOSO, ACOMODA MUITO BEM A GARRAFA DE SUCO E A MARMITINHA DO LANCHE. PRODUTO ENTREGUE ANTES DO PRAZO PREVISTO. 
1 5 Produto foi entregue dentro do prazo e foi mais rápido do que eu pensava! Amei!! 
1 5 Minha esposa amou.
1 5 adorei
1 5 A maioridade das cameras de espiã em baixo de R$100 são lixo. Raramente funcionam, e com qualidade péssima de video e som. Investi um pouco mais aqui e fiquei impressionado com im

1 2 Minha casa fica a 100 metros dos correios.Disseram que a casa estava fechada ma não me ligaram apesar de ter meus números.
1 2 Entrega foi rápida , porém o produto de péssima qualidade, fino , não passou nem perto do que eu esperava , está certo que foi barato mas eu não pagaria nem 10,00.
1 2 Discrição de panela de ferro não conferi 
1 2 Comprei um produto, recebi outro completamente diferente, ja abri uma reclamação, estou aguardando retorno.
1 2 Falta entregar um dos produtos 
1 2 Comprei um relógio Casio e veio riscado! Sim, riscado! Fiquei muito triste, porque além de ter pagado caro o produto veio danificado. E outra, paguei 68,00 de frete! Um absurdo isso. Nunca mais!
1 2 COMPREI DOIS SO ENTREGARAM 1
1 2 A cor e diferente de como está na foto
1 2 Muitas rebarbas na hélice do produto, risco de corte/ acidente ao manusear. Se soubesse não teria feito a aquisição.
1 2 Comprei 3 cabos RCA + 1 suporte de celular.
Só me foi entregue 1 cabo RCA.
1 2 Pela indicação achei que peça se

1 4 A panela veio com o tamanho maior.Mas tudo bem.Fico com ela.Gostaria que arrumassem o meu nome pra envio.Ronaldo é meu cunhado eu sou Magda B M Santos. Quero receber mercadorias no meu nome.Obrigada.
1 4 Produtos excelente 
1 4 gostaria de saber de houve o debito correto no meu cartão. tive cartão clonado. 
1 4 Gostei chegou rapidinho
1 4 Tudo certinho conforme divulgação, o produto chega sem surpresas.
1 4 amei o produto veio perfeito
1 4 produto conforme anuncio, aguardar quanto aos resultados
1 4 Ate momento produto funcionando
1 4 Comprei o telefone mas veio com uma cor diferente comprei um azul e preto
1 4 produto bom, chegou no prazo
1 4 Achei, bom gostei recomendo!
1 4 Super indico 
1 4 o endereço veio faltando dados com isso dificultou a entrega no condomínio (veio faltando dados do Bloco e Sala)
1 4 O produto é bom! Tem uma qualidade media, mais pra alta, recomendo pra iniciar e também pra decorativo e perfeito, vem com o suporte pra colocar na parede. 
1 4 Houve um proble

In [11]:
correct, incorrect

(1770, 5741)

1 4 Mosquitos frustrados pela boa qualidade do Mosquitero. Nunca mais acordar cantando.
1 4 A base é pequena. Não sei se um bauleto ficará firme.
5 4 Recomendo a todos!
5 4 Tudo dentro do previsto 
5 4 Bons panos de pratos !
5 4 A entrega até foi antes do previsto..gostei muito
1 4 Só posso avaliar quando receber o produto. Ainda está no prazo de entrega informado. 
1 4 Solicito a evitarem o envio de tanta papelada de propaganda junto, em muito repetitivos a cada compra. 
Lembrem-se: NÃO HÁ COMO SE TER PAZ SEM ECOLOGIA.

In [12]:
bias = dict()
for p in pred:
    try:
        bias[p]+=1
    except:
        bias[p]=1

In [13]:
bias

{1: 7511}